In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pathlib
import swifter
from functools import reduce

from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

import geojson
import h3

from geovoronoi import voronoi_regions_from_coords
crs4326 = {'init': 'epsg:4326'}

In [2]:
DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../../..' )

influences_df = pd.read_csv(DIR_ROOT + '/data/04_itinerary/influences.csv')
influences_df = influences_df[influences_df['domestic'] == True]

localities = pd.read_csv(DIR_ROOT + '/data/02_processed/localities.csv')
localities['geometry'] = localities['geometry'].apply(wkt.loads)
localities = gpd.GeoDataFrame(localities, crs=crs4326).set_geometry('geometry')

In [3]:
bbox = localities.buffer(1).total_bounds
bounds = dict(geojson.Polygon([[[bbox[0], bbox[1]], [bbox[0], bbox[3]], [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]))
hex_ids = list(h3.polyfill(bounds, 4))

hex_df = gpd.GeoDataFrame(
    [{"geometry": Polygon(h3.h3_to_geo_boundary(x)), "id": x} for x in hex_ids], crs=crs4326
)
hex_df.set_index('id', inplace=True)

hex_df['id'] = hex_df.index
hex_df['geometry'] = hex_df.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x['id'])), axis=1)
hex_df

<ipython-input-3-7dfe25777e9d>:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox = localities.buffer(1).total_bounds


,geometry,id
id,,
847a90dffffffff,"POLYGON ((7.15652 51.21265, 7.34660 51.27283, ...",847a90dffffffff
847ad67ffffffff,"POLYGON ((8.25103 44.80135, 8.44641 44.87207, ...",847ad67ffffffff
845221bffffffff,"POLYGON ((22.21724 45.07354, 22.00896 44.94325...",845221bffffffff
8452731ffffffff,"POLYGON ((18.40947 48.75635, 18.21037 48.62957...",8452731ffffffff
846395bffffffff,"POLYGON ((11.11608 51.05574, 10.92915 50.93759...",846395bffffffff
...,...,...
844386dffffffff,"POLYGON ((21.96025 54.77503, 21.76004 54.85821...",844386dffffffff
847a9ebffffffff,"POLYGON ((8.07965 49.10792, 8.26998 49.17151, ...",847a9ebffffffff
84526c9ffffffff,"POLYGON ((20.73144 52.95762, 20.54046 52.82805...",84526c9ffffffff


In [4]:
def get_influence_origins(hex_geo):
    localities_in_hex = localities[localities.intersects(hex_geo)]

    infl_list = []
    for lihi, locality_in_hex in localities_in_hex.iterrows():
        inf_object_rows = influences_df[influences_df['name'] == locality_in_hex['name']]
        infl_list.extend(inf_object_rows['locality'].tolist())

    return { x: infl_list.count(x) for x in infl_list}

hex_df['infl'] = hex_df.apply(
    lambda x: get_influence_origins(x['geometry']), 
    axis=1
)
print(hex_df['infl'].tolist())

hex_df['all'] = hex_df.apply(
    lambda x: sum(x['infl'].values()), 
    axis=1
)

centers_to_check = ['Praha', 'Brno', 'Wien', 'Znojmo', 'Graz', 'Krems an der Donau', 'Linz']


for center in centers_to_check:
    hex_df[center] = hex_df.apply(
        lambda x: x['infl'][center] if center in x['infl'] else 0, 
        axis=1
    )

hex_df['others'] = hex_df.apply(
    lambda x: x['all'] - sum(x[center] for center in centers_to_check) , 
    axis=1
)



[{}, {}, {}, {}, {}, {}, {}, {'Znojmo': 1, 'Praha': 2}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Praha': 1, 'Písek': 1, 'Brno': 1}, {}, {'Wien': 18, 'Głubczyce': 1, 'Znojmo': 2, 'Sitzenberg': 2, 'Poděbrady': 1, 'Praha': 3, 'Jihlava': 2, 'Jaroměřice nad Rokytkou': 1, 'Zbraslav': 1, 'Brno': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {'Krems an der Donau': 3, 'Kleinweikersdorf': 1, 'Zwettl': 2, 'Brno': 1, 'Wien': 2}, {}, {'Znojmo': 2, 'Brno': 1, 'Korneuburg': 1, 'Wien': 2, 'Písek': 1, 'Praha': 1}, {'Praha': 5, 'Kravaře u České Lípy': 1, 'Jablonné v Podještědí': 1, 'Budyně nad Ohří': 2, 'Litoměřice': 1}, {}, {}, {}, {}, {}, {'Kladruby': 2, 'Praha': 5, 'Křivoklát': 2, 'Plasy': 5}, {}, {}, {}, {}, {}, {}, {'Jihlava': 1}, {'Hainburg': 1}, {}, {}, {}, {'Most': 2, 'Praha': 6, 'Poděbrady': 1, 'Wien': 2, 'Křivoklát': 2}, {'Brno': 1, 'Jindřichův Hradec': 1, 'Praha': 1, 'Uničov': 1, 'Hradec nad Moravicí': 2}, {}, {'Praha': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}

In [5]:

hex_df.reset_index(drop=True).to_file(DIR_ROOT + '/data/06_outputs/hex_influence.geojson', driver="GeoJSON")

hex_df_geo = hex_df.copy()
hex_df_geo['centroid'] = hex_df_geo.apply(
    lambda x: x['geometry'].centroid, 
    axis=1
)
hex_df_geo = hex_df_geo.drop(columns=["geometry"])
hex_df_geo.set_geometry('centroid', inplace=True)
hex_df_geo = hex_df_geo.reset_index(drop=True)
hex_df_geo.to_file(DIR_ROOT + '/data/06_outputs/hex-centroids_influence.geojson', driver="GeoJSON")
